<h1>Interactive Saliency map visualization</h1>

<h2>Introduction</h2>

<p>This notebook help to visualize saliency maps from the npz files.</p>
<p>Please check that the <b>requirements.txt</b> file is installed, that the ILSVRC2012 validation set <b>images</b> are in the "images" folder and .npz files in the <b>npz</b> folder (files available to download at <a href=https://polycam.ddns.net>https://polycam.ddns.net</a>, view <b>README</b> file for more informations). This notebook in addition requires <b>ipywidgets</b></p>
<p>Run all cells at once, then you can use the interactive interface. If you change any parameter you should restart all cells</p>

<h2>Parameters</h2>

<h3>Saliency selection</h3>
<p>If you generated npz of the saliency maps yourself from the script for only some of the above methods and models, or if you only extracted part of the npz zip files, please <b>comment the unused saliency methods</b> bellow.</p>

In [ ]:
model_list = ['vgg16', 'resnet50']

saliency_list = []
saliency_list += ['pcamp4', 'pcamm4', 'pcampm4'] # Poly-CAM
# Uncomment the following line if you want to add maps from the ablation study (Poly-CAM without LNorm)
#saliency_list += ['pcampnolnorm4', 'pcammnolnorm4', 'pcampm4nolnorm']
saliency_list.append('zoomcam') # Zoom-CAM
saliency_list.append('layercamfusion4') # Layer-CAM
saliency_list.append('gradcam') # Grad-CAM
saliency_list.append('gradcampp') # Grad-CAM++
saliency_list.append('smoothgradcampp') # SmoothGrad-CAM++
saliency_list.append('scorecam') # Score-CAM
saliency_list.append('sscam') # SS-CAM
saliency_list.append('iscam') # IS-CAM
saliency_list.append('ixg') # InputXGradient
saliency_list.append('ig') # IntegratedGradient
saliency_list.append('sg') # SmoothGrad
saliency_list.append('occlusion') # Occlusion
saliency_list.append('rise') # RISE

<h3>Other parameters</h3>

In [ ]:
image_list = 'images.txt'
labels_list = 'imagenet_validation_imagename_labels.txt'
image_folder = 'images'
npz_folder = './npz'
results_folder = './results'
input_size = 224

<p>Please reload cells bellow if any parameter above is changed</p>

<h2>Code</h2>

<h3>Imports and load files</h3>

In [ ]:
import torch
import torch.nn.functional as F

from torchvision import transforms

import numpy as np

from benchmarks.imagesDataset import ImagesDataset
from benchmarks.utils import imagenet_labels, overlay

from matplotlib import pyplot as plt

import pandas as pd

from IPython.display import display
import ipywidgets as widgets

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((input_size, input_size))
                                ])


dataset = ImagesDataset(image_list, labels_list, image_folder, transform=transform)

In [ ]:
saliency_file_dict = {}
results_dict = {}
for model in model_list:
    saliency_file_dict[model] = {}
    results_df = pd.read_csv(results_folder + "/" + model + "_results.csv", header = None, index_col=False)
    results_dict[model] = dict(zip(results_df.iloc[:,0], results_df.iloc[:,1]))
    for saliency in saliency_list:
        saliency_npz = npz_folder + '/' + model + "_" + saliency + ".npz"
        saliency_file_dict[model][saliency] = np.load(saliency_npz)

#saliency_npz = npz_folder + '/' + model.value + "_" + saliency.value + ".npz"
# get saliencies from file
#saliency_file = np.load(saliency_npz)



In [ ]:
def display_overlayed(x=None):
    with output_figure:
        image, _, name = dataset[image_slider.value]
        image = image.unsqueeze(0)
        saliency_map = torch.tensor(saliency_file_dict[model.value][saliency.value][dataset[image_slider.value][2]])
        sh, sw = saliency_map.shape[-2:]
        if saliency.value in ["ig", "ixg"]:
            saliency_map = torch.abs(saliency_map)
        saliency_map = ( saliency_map - saliency_map.min() ) / ( saliency_map.max() - saliency_map.min() )
        saliency_map = saliency_map.view(1, 1, sh, sw)
        saliency_map = F.interpolate(saliency_map, dataset[image_slider.value][0].shape[-2:], mode='bilinear').type(torch.float32)
        fig = plt.figure(figsize=(8,8))
        plt.imshow(overlay(dataset[image_slider.value][0], saliency_map[0].detach(), alpha=alpha_slider.value, colormap="turbo" ))
        plt.axis('off')
        output_figure.clear_output()
        fig.suptitle("saliency map for top class on " + model.value + ":\n " + imagenet_labels[results_dict[model.value][name]], fontsize=24)
        display(fig)
        plt.close()
    return None
        

In [ ]:
saliency = widgets.Dropdown(
    options=saliency_list,
    description='Saliency',
    disabled=False,
)


model = widgets.Dropdown(
    options=model_list,
    description='Model:',
    disabled=False,
)

image_slider = widgets.IntSlider(
    value=1000,
    min=0,
    max=1999,
    step=1,
    description='Image selection:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)

alpha_slider = widgets.FloatSlider(
    value=0.4,
    min=0,
    max=1.0,
    step=0.1,
    description='Alpha:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',)

output_figure = widgets.Output()
output_figure.layout.height = '500px'

saliency.observe(display_overlayed, "value")
model.observe(display_overlayed, "value")
image_slider.observe(display_overlayed, "value")
alpha_slider.observe(display_overlayed, "value")



menu=widgets.VBox([saliency,
                   model,
                   image_slider,
                   alpha_slider
                  ])

app_layout = widgets.Layout(display='flex',
                flex_flow='row nowrap',
                align_items='center',
                border='none',
                width='100%',
                margin='5px 5px 5px 5px')

app=widgets.HBox([menu, output_figure], layout=app_layout)

display_overlayed()

<h2>Saliency overlay</h2>

In [ ]:
display(app)